In [1]:
import os, sys
import geopandas as gpd
import pandas as pd
from geopy.geocoders import GeoNames, Nominatim
from shapely.geometry import Point

In [2]:
iso3 = 'LBR'

In [3]:
input_dir = "/home/public/Data/PROJECTS/Health" #
out_folder = os.path.join(input_dir, "output", iso3)
# out_folder = r"C:\repos\health-equity-access\notebooks\output"
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

### Destinations


In [4]:
# lbr_master = pd.read_excel(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia.xlsx"))
lbr_master = pd.read_csv(os.path.join(input_dir, "from_tashrik", "master lists", "LIberia MFL Adjusted.csv"))

In [5]:
lbr_master[:3]

,facility,admin_county,admin_district,today,latitude,longitude,_getcoordinate_altitude,precision,facilitytype,ownership,...,offer_imci,offer_essentialnutrition,offer_fpcouple,offer_fpcounsel,offer_adolescentsrhr,offer_sexgender,offer_detectmanage,offer_precenttreat,dup,update_date
0,Abc Clinic,Montserrado,Somalia Drive,2021-08-07,6.331593,-10.743366,4.009424,4.983,clinic,Private (non-faith based),...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,07aug2021
1,Acfi Medical Clinic,Montserrado,Central Monrovia,2021-10-19,6.304997,-10.799709,2.900000,4.978,clinic,"Private, (faith based)",...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,19oct2021
2,Acres Clinic,Montserrado,St Paul River,2021-11-18,6.412980,-10.778231,-2.712283,4.933,clinic,Private (non-faith based),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,18nov2021


In [6]:
lbr_master.functional.value_counts()

functional                     872
Can't confirm at the moment      1
Name: functional, dtype: int64

In [7]:
lbr_master.columns

Index(['facility', 'admin_county', 'admin_district', 'today', 'latitude',
       'longitude', '_getcoordinate_altitude', 'precision', 'facilitytype',
       'ownership', 'opendate', 'functional', 'offer_anc', 'offer_obstetric',
       'offer_emonc', 'offer_postpartum', 'offer_newborncare',
       'offer_maternalnewboen', 'offer_csection', 'offer_childimmune',
       'offer_imci', 'offer_essentialnutrition', 'offer_fpcouple',
       'offer_fpcounsel', 'offer_adolescentsrhr', 'offer_sexgender',
       'offer_detectmanage', 'offer_precenttreat', 'dup', 'update_date'],
      dtype='object')

In [8]:
lbr_master.facilitytype.value_counts()

clinic           750
health center     88
hospital          24
health post        6
schoold based      4
SDP                1
Name: facilitytype, dtype: int64

In [9]:
lbr = lbr_master.loc[lbr_master.functional=="functional"].copy()

In [10]:
len(lbr)

872

In [11]:
lbr_missing = lbr.loc[lbr.latitude.isna()].copy()

In [12]:
lbr_non_missing = lbr.loc[~(lbr.latitude.isna())].copy()

In [13]:
len(lbr_missing), len(lbr_non_missing)

(55, 817)

In [14]:
lbr_master_old = pd.read_excel(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia.xlsx"))
lbr_old = lbr_master_old.loc[lbr_master_old.Status!="non-functional"].copy()
lbr_old = lbr_old.loc[~(lbr_old.Lat.isna())].copy()

In [16]:
# lbr_old.loc[lbr_old['HF Name']=="Bellemu Clinic", "Lat"]

In [17]:
for idx, row in lbr_missing.iterrows():
#     print(f"Geocoding {idx}, {i} of {len(lbr_missing)}")
    # Check if there ia another entry with location
    lbr_copy = lbr_old.loc[lbr_old['HF Name']==row.facility,['Lat', 'Long']]
    if len(lbr_copy)>0:
        res = lbr_copy.iloc[0]
        lbr_missing.loc[idx, "longitude"] = res.Long
        lbr_missing.loc[idx, "latitude"] = res.Lat
#         print("\tSuccess for Another facility with same Location")
        lbr_missing.loc[idx, "geocoding_method"] = "Location from old Master List"
    else:
        lbr_missing.loc[idx, "geocoding_method"] = "None"

In [18]:
lbr_missing.geocoding_method.value_counts()

Location from old Master List    43
None                             12
Name: geocoding_method, dtype: int64

In [20]:
# lbr_missing.loc[lbr_missing.geocoding_method=="None"]

In [21]:
lbr_io = gpd.read_file(os.path.join(input_dir, "healthsitesio", "Liberia-node.shp"))

In [22]:
lbr_io.loc[lbr_io.name=="Gbeapo Health Center"].geometry.iloc[0].x

-8.013713282055647

In [23]:
for idx, row in lbr_missing.loc[lbr_missing.geocoding_method=="None"].iterrows():
#     print(f"Geocoding {idx}, {i} of {len(lbr_missing)}")
    # Check if there ia another entry with location
    lbr_copy = lbr_io.loc[lbr_io['name']==row.facility,['geometry']]
    if len(lbr_copy)>0:
        res = lbr_copy.iloc[0]
        lbr_missing.loc[idx, "longitude"] = res.geometry.x
        lbr_missing.loc[idx, "latitude"] = res.geometry.y
#         print("\tSuccess for Another facility with same Location")
        lbr_missing.loc[idx, "geocoding_method"] = "Location from Health Sites IO"
    else:
        lbr_missing.loc[idx, "geocoding_method"] = "None"

In [24]:
lbr_missing.geocoding_method.value_counts()

Location from old Master List    43
None                             10
Location from Health Sites IO     2
Name: geocoding_method, dtype: int64

In [25]:
# counties_gpd = gpd.read_file(r"C:\repos\health-equity-access\notebooks\input\lbr_admbnda_adm1_ocha_2019.shp")
counties_gpd = gpd.read_file(os.path.join(input_dir, 'gadm', 'LBR', 'lbr_admbnda_adm1_ocha_20191104.shp'))

In [26]:
geolocator = Nominatim(user_agent="health-equity")

df = lbr_missing.loc[lbr_missing.geocoding_method=="None"].copy()
df.loc[:, "location"] = df.loc[:, "facility"]
county_array = lbr_missing['admin_county'].values

for idx, row in df.iterrows():
    
    if "Community" in row['location']:
        df.loc[idx, "location"] = row['location'].replace(" Community", "")
    if "Clinic" in row['location']:
        df.loc[idx, "location"] = row['location'].replace(" Clinic", "")
        
    try:
        res = geolocator.geocode(row['location'] + ', ' + row['admin_county'], country_codes='lr')
        # if within county
        if counties_gpd.loc[counties_gpd['ADM1_EN'] == row['admin_county']].contains(Point(res.longitude,res.latitude)).values[0] == True:
            lbr_missing.loc[idx, "longitude"] = res.longitude
            lbr_missing.loc[idx, "latitude"] = res.latitude
            print("\tSuccess for Facility name and County query")
            lbr_missing.loc[idx, "geocoding_method"] = "Facility name and county query"
    except:
        lbr_missing.loc[idx, "geocoding_method"] = "None"

In [27]:
lbr_missing.geocoding_method.value_counts()

Location from old Master List    43
None                             10
Location from Health Sites IO     2
Name: geocoding_method, dtype: int64

In [31]:
len(lbr_missing)

55

In [28]:
geolocator = Nominatim(user_agent="health-equity")

df = lbr_missing.loc[lbr_missing.geocoding_method=="None"].copy()
df.loc[:, "location"] = df.loc[:, "facility"]
county_array = lbr_missing['admin_county'].values

for idx, row in df.iterrows():
    
    if "Community" in row['location']:
        df.loc[idx, "location"] = row['location'].replace(" Community", "")
    if "Clinic" in row['location']:
        df.loc[idx, "location"] = row['location'].replace(" Clinic", "")
        
    try:
        res = geolocator.geocode(row['admin_district'] + ', ' + row['admin_county'], country_codes='lr')
        # if within county
        if counties_gpd.loc[counties_gpd['ADM1_EN'] == row['admin_county']].contains(Point(res.longitude,res.latitude)).values[0] == True:
            lbr_missing.loc[idx, "longitude"] = res.longitude
            lbr_missing.loc[idx, "latitude"] = res.latitude
#             print("\tSuccess for District name and County query")
            lbr_missing.loc[idx, "geocoding_method"] = "District name and county query"
    except:
        lbr_missing.loc[idx, "geocoding_method"] = "None"

	Success for District name and County query
	Success for District name and County query
	Success for District name and County query
	Success for District name and County query
	Success for District name and County query
	Success for District name and County query
	Success for District name and County query
	Success for District name and County query
	Success for District name and County query
	Success for District name and County query


In [29]:
lbr_missing.geocoding_method.value_counts()

Location from old Master List     43
District name and county query    10
Location from Health Sites IO      2
Name: geocoding_method, dtype: int64

In [30]:
lbr_missing.to_csv(os.path.join(out_folder, "Liberia MFL Adjusted Geocoded.csv"))